In [6]:
#Importing essential libraries and modules
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D,Input,Dense,Concatenate,Dropout,Flatten
from tensorflow.keras.applications import VGG16,ResNet50,DenseNet121

In [2]:
from google.colab import files
kaggle=files.upload()

Saving kaggle.json to kaggle.json


In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [4]:
!kaggle datasets download -d salader/dogs-vs-cats
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
100% 1.06G/1.06G [00:05<00:00, 204MB/s]
100% 1.06G/1.06G [00:05<00:00, 208MB/s]


In [7]:
Inputs = Input(shape=(224, 224, 3))
vgg16=VGG16(include_top=False,weights="imagenet",input_tensor=Inputs)
resnet=ResNet50(include_top=False,weights="imagenet",input_tensor=Inputs)
densenet=DenseNet121(include_top=False,weights="imagenet",input_tensor=Inputs)
for layer in vgg16.layers:
    layer.name = "vgg_" + layer.name
for layer in resnet.layers:
    layer.name = "resnet_" + layer.name
for layer in densenet.layers:
    layer.name = "densenet_" + layer.name


In [8]:
for model in [vgg16,resnet,densenet]:
  model.trainable=False
vgg_out=GlobalAveragePooling2D()(vgg16.output)
resnet_out=GlobalAveragePooling2D()(resnet.output)
densenet_out=GlobalAveragePooling2D()(densenet.output)

In [9]:
#Functional API
from tensorflow.keras import Model
concat=Concatenate()([vgg_out,resnet_out,densenet_out])
x=concat
x=Dense(128,activation="relu")(x)
output=Dense(1,activation="sigmoid")(x)
ensemble_model=Model(inputs=Inputs,outputs=output)
ensemble_model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])


In [10]:
# generators
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory = '/content/train',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(224,224)
)

validation_ds = tf.keras.utils.image_dataset_from_directory(
    directory = '/content/test',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(224,224)
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [11]:
# Normalize
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [ ]:
ensemble_model.fit(train_ds,epochs=4,validation_data=validation_ds)

Epoch 1/4
625/625 ━━━━━━━━━━━━━━━━━━━━ 295s 473ms/step - accuracy: 0.9818 - loss: 0.0467 - val_accuracy: 0.9808 - val_loss: 0.0472
Epoch 2/4
625/625 ━━━━━━━━━━━━━━━━━━━━ 284s 412ms/step - accuracy: 0.9868 - loss: 0.0364 - val_accuracy: 0.9788 - val_loss: 0.0543
Epoch 3/4
625/625 ━━━━━━━━━━━━━━━━━━━━ 263s 414ms/step - accuracy: 0.9878 - loss: 0.0315 - val_accuracy: 0.9812 - val_loss: 0.0470
Epoch 4/4
625/625 ━━━━━━━━━━━━━━━━━━━━ 293s 463ms/step - accuracy: 0.9889 - loss: 0.0304 - val_accuracy: 0.9808 - val_loss: 0.0512


In [12]:
#Stacking using meta model
vgg16_base=VGG16(input_tensor=Inputs,include_top=False)
resnet_base=ResNet50(input_tensor=Inputs,include_top=False)
densenet_base=DenseNet121(input_tensor=Inputs,include_top=False)
vgg16_out=Flatten()(vgg16_base.output)
resnet_out=Flatten()(resnet_base.output)
densenet_out=Flatten()(densenet_base.output)
vgg16_out=Dense(1,activation="sigmoid")(vgg16_out)
densenet_out=Dense(1,activation="sigmoid")(densenet_out)
resnet_out=Dense(1,activation="sigmoid")(resnet_out)
vgg_model=Model(inputs=Inputs,outputs=vgg16_out)
vgg_model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
resnet_model=Model(inputs=Inputs,outputs=resnet_out)
resnet_model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
densenet_model=Model(inputs=Inputs,outputs=densenet_out)
densenet_model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
for layer in vgg16_base.layers:
    layer.name = "vggbase_" + layer.name
for layer in resnet_base.layers:
    layer.name = "resnetbase_" + layer.name
for layer in densenet_base.layers:
    layer.name = "densenetbase_" + layer.name
for model in [vgg16_base,resnet_base,densenet_base]:
  model.trainable=False

In [13]:
vgg_model.fit(train_ds,epochs=2,validation_data=validation_ds)
resnet_model.fit(train_ds,epochs=2,validation_data=validation_ds)
densenet_model.fit(train_ds,epochs=2,validation_data=validation_ds)

Epoch 1/2
  1/625 ━━━━━━━━━━━━━━━━━━━━ 3:32:24 20s/step - accuracy: 0.6562 - loss: 0.8071

KeyboardInterrupt: 

In [ ]:
vgg_pred=vgg_model.predict(validation_ds)
resnet_pred=resnet_model.predict(validation_ds)
densenet_pred=densenet_model.predict(validation_ds)

157/157 ━━━━━━━━━━━━━━━━━━━━ 26s 162ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 21s 102ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 34s 144ms/step


In [ ]:
y = np.array([y.numpy() for _, y in validation_ds.unbatch()])
y.shape

(5000,)

In [ ]:
x=np.concatenate((vgg_pred,resnet_pred,densenet_pred),axis=1)
x.shape

(5000, 3)

In [ ]:
#Meta model
import sklearn
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y)
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
from sklearn.metrics import classification_report
report=classification_report(y_test,y_pred)

In [ ]:
print(report)

              precision    recall  f1-score   support

           0       0.49      0.37      0.43       643
           1       0.47      0.59      0.53       607

    accuracy                           0.48      1250
   macro avg       0.48      0.48      0.48      1250
weighted avg       0.48      0.48      0.47      1250



In [ ]:
#Instaed of using meta model
y_pred=[]
for i in x:
  if (i[0]>0.5 and i[1]>0.5)or(i[1]>0.5 and i[2]>0.5)or(i[2]>0.5 and i[0]>0.5):
    y_pred.append(1)
  else:
    y_pred.append(0)
print(classification_report(y,y_pred))


              precision    recall  f1-score   support

           0       0.51      0.60      0.55      2500
           1       0.51      0.42      0.46      2500

    accuracy                           0.51      5000
   macro avg       0.51      0.51      0.51      5000
weighted avg       0.51      0.51      0.51      5000

